In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using StatsBase
include("utils.jl")
include("kmeans_moments_lattice.jl")
using .LatticeGeneration
using JLD2
using SDDP, HiGHS, Plots, XLSX, Mustache, PrettyTables

In [2]:
includet("goal_investing_sddp_model.jl")
using .GoalInvestingSDDP

In [19]:
scenario_grid = load_object("scenario_grid.jld2")
println(size(scenario_grid.probs))
println(size(scenario_grid.states))

(16,)
(15, 17, 3)


In [20]:
graph = SDDP.MarkovianGraph(scenario_grid.probs[1:end-1])

Root
 (0, 1)
Nodes
 (1, 1)
 (2, 1)
 (2, 2)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (2, 7)
 (2, 8)
 (2, 9)
 (2, 10)
 (2, 11)
 (2, 12)
 (2, 13)
 (2, 14)
 (2, 15)
 (2, 16)
 (2, 17)
 (3, 1)
 (3, 2)
 (3, 3)
 (3, 4)
 (3, 5)
 (3, 6)
 (3, 7)
 (3, 8)
 (3, 9)
 (3, 10)
 (3, 11)
 (3, 12)
 (3, 13)
 (3, 14)
 (3, 15)
 (3, 16)
 (3, 17)
 (4, 1)
 (4, 2)
 (4, 3)
 (4, 4)
 (4, 5)
 (4, 6)
 (4, 7)
 (4, 8)
 (4, 9)
 (4, 10)
 (4, 11)
 (4, 12)
 (4, 13)
 (4, 14)
 (4, 15)
 (4, 16)
 (4, 17)
 (5, 1)
 (5, 2)
 (5, 3)
 (5, 4)
 (5, 5)
 (5, 6)
 (5, 7)
 (5, 8)
 (5, 9)
 (5, 10)
 (5, 11)
 (5, 12)
 (5, 13)
 (5, 14)
 (5, 15)
 (5, 16)
 (5, 17)
 (6, 1)
 (6, 2)
 (6, 3)
 (6, 4)
 (6, 5)
 (6, 6)
 (6, 7)
 (6, 8)
 (6, 9)
 (6, 10)
 (6, 11)
 (6, 12)
 (6, 13)
 (6, 14)
 (6, 15)
 (6, 16)
 (6, 17)
 (7, 1)
 (7, 2)
 (7, 3)
 (7, 4)
 (7, 5)
 (7, 6)
 (7, 7)
 (7, 8)
 (7, 9)
 (7, 10)
 (7, 11)
 (7, 12)
 (7, 13)
 (7, 14)
 (7, 15)
 (7, 16)
 (7, 17)
 (8, 1)
 (8, 2)
 (8, 3)
 (8, 4)
 (8, 5)
 (8, 6)
 (8, 7)
 (8, 8)
 (8, 9)
 (8, 10)
 (8, 11)
 (8, 12)
 (8, 13)
 

In [36]:
data_source = DataFrame(XLSX.readtable("./goal_data.xlsx", "goal_data_single_goal")) 
data = round.(data_source[1:16,:],digits=3)

Row,t,minimum_limit,acceptable_limit,desired_limit,inflows,priority,minimum_utility,acceptable_utility,desired_utility,above_desired_utility,cashflows
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,6.0,8.0,9.0,0.0,0.067,0.278,0.167,0.067,0.001,-9.0
2,2.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
3,3.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
4,4.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
5,5.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
6,6.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
7,7.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
8,8.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
9,9.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0


In [37]:
model_data = GoalsData3(
    data.minimum_limit,
    data.acceptable_limit,
    data.desired_limit,
    data.inflows,
    data.minimum_utility,
    data.acceptable_utility,
    data.desired_utility,
    data.above_desired_utility,
    35.0,
    0.0029
    )

GoalsData3([6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 81.0], [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 128.0], [9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 159.0], [0.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0], Float32[0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.309], Float32[0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.106], Float32[0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.032], Float32[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 35.0, 0.0029f0)

In [38]:
model = asset_management_alm(model_data, scenario_grid)

A policy graph with 256 nodes.
 Node indices: (1, 1), ..., (16, 17)


In [39]:
SDDP.train(model; iteration_limit = 400, log_frequency = 50)


-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 256
  state variables : 3
  scenarios       : 2.86242e+18
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [18, 18]
  AffExpr in MOI.EqualTo{Float64}         : [5, 5]
  VariableRef in MOI.GreaterThan{Float64} : [15, 15]
  VariableRef in MOI.LessThan{Float64}    : [3, 4]
numerical stability report
  matrix range     [8e-01, 1e+00]
  objective range  [1e-03, 1e+00]
  bounds range     [1e+00, 8e+01]
  rhs range        [1e+01, 4e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-----------------------------------------

In [41]:
result = simulate_policy(model)

Main.GoalInvestingSDDP.Optimistion_Result(-60.0619, -60.0294, 0.0917, [8.0 8.0 … 9.0 162.88640884722153; 8.0 8.0 … 9.0 203.22030509223458; … ; 8.0 8.0 … 8.0 130.09333779422533; 8.0 8.0 … 8.0 88.93159612611889], [26.921926295638798 26.921926295638798 … 26.921926295638798 26.921926295638798; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 36.98363408241602 33.27026210484489 … 36.56276372500172 35.6145241514394; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 45.69494753368981 … 37.614365215141326 43.30990963097261; 50.84644647190558 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 47.572266138908745 54.20487428838944 … 64.87011134956867 64.62572500929582; 28.913292938773566 0.0 … 40.97262249204926 0.0; 79.08267804343042 103.52738033280795 … 0.0 0.0;;; 168.33991840380486 175.18333300907656 … 100.13109943185076 83.66661903050752; 0.03572352585997038 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [42]:
# 3y returns
result.lower_bound

-60.0619

In [43]:
result.lower_bound

-60.0619

In [28]:

function calculate_perc(data, perc = [0.05, 0.25, 0.5, 0.75, 0.95])
    n_scenarios, n_stages = size(data)
    
    result = zeros(n_stages, length(perc)) 

    for t in 1:n_stages
       result[t,:] = quantile(data[:,t], perc)
    end

    return DataFrame(round.(result,digits=2),string.(perc))
  
end

calculate_perc (generic function with 2 methods)

In [44]:
calculate_perc(result.consumption_dist)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,8.0,8.0,8.0,8.0,8.0
2,8.0,8.0,8.0,8.0,8.0
3,8.0,8.0,8.0,8.0,9.0
4,8.0,8.0,8.0,8.28,9.0
5,8.0,8.0,8.0,9.0,9.0
6,8.0,8.0,8.0,9.0,9.0
7,8.0,8.0,8.0,9.0,9.0
8,8.0,8.0,8.67,9.0,9.0
9,8.0,8.0,9.0,9.0,9.0


In [45]:
# ret 3y

calculate_perc(result.consumption_dist)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,8.0,8.0,8.0,8.0,8.0
2,8.0,8.0,8.0,8.0,8.0
3,8.0,8.0,8.0,8.0,9.0
4,8.0,8.0,8.0,8.28,9.0
5,8.0,8.0,8.0,9.0,9.0
6,8.0,8.0,8.0,9.0,9.0
7,8.0,8.0,8.0,9.0,9.0
8,8.0,8.0,8.67,9.0,9.0
9,8.0,8.0,9.0,9.0,9.0


In [46]:
# ret 3y
calculate_perc(result.assets_dist[1,:,:])


Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,26.92,26.92,26.92,26.92,26.92
2,11.1,29.33,31.51,35.61,38.71
3,14.69,33.23,38.01,42.66,47.67
4,21.17,35.17,41.09,47.1,54.88
5,22.51,39.41,46.4,52.85,61.67
6,25.94,40.63,48.92,57.66,68.01
7,28.46,41.95,53.27,62.27,75.62
8,32.79,45.01,56.59,67.17,81.76
9,32.97,53.68,64.06,74.57,91.1


In [47]:
wealth = dropdims(sum(result.assets_dist, dims=1), dims=1)
calculate_perc(wealth)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,26.92,26.92,26.92,26.92,26.92
2,27.99,30.95,33.27,36.88,41.67
3,31.35,36.06,39.43,43.57,49.22
4,34.73,40.65,45.36,49.93,59.03
5,37.95,45.18,50.77,56.43,68.32
6,41.46,49.37,56.7,63.87,76.9
7,44.32,53.99,62.28,71.28,86.8
8,47.74,58.74,68.05,78.4,98.72
9,50.9,63.62,74.04,86.18,108.02


In [55]:
calculate_perc(result.assets_dist[1] ./ wealth)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0
2,0.65,0.73,0.81,0.87,0.96
3,0.55,0.62,0.68,0.75,0.86
4,0.46,0.54,0.59,0.66,0.78
5,0.39,0.48,0.53,0.6,0.71
6,0.35,0.42,0.47,0.55,0.65
7,0.31,0.38,0.43,0.5,0.61
8,0.27,0.34,0.4,0.46,0.56
9,0.25,0.31,0.36,0.42,0.53


In [34]:
report = print_report(result, model_data, "c:\\Users\\matsz\\programowanie\\Optymalizacja_portfela\\julia_msp\\reports\\tpl.html")

MethodError: MethodError: no method matching render(::String, ::Main.GoalInvestingSDDP.Optimistion_Result; TITLE::String, input_data::GoalsData3)

Closest candidates are:
  render(::AbstractString, ::Any; tags) got unsupported keyword arguments "TITLE", "input_data"
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:55
  render(!Matched::Mustache.MustacheTokens, ::Any) got unsupported keyword arguments "TITLE", "input_data"
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:30
  render(::AbstractString; kwargs...)
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:56


- simulate policy
- generate report summary
- save report